<img src="assets/logo_light_or_dark_mode.svg" justify-content="center">

# Extracting information from log files - Introducing "gurobi-logtools"

When running Gurobi, you will almost always create a log file. However, this name does not do it justice since the log file contains a lot of run time information about Gurobi which is critical to understand its behaviour and how to fine tune it.



Consider the following excerpt from a log file:

```plain
   Nodes    |    Current Node    |     Objective Bounds      |     Work
Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time
[...]
   460   223 infeasible   15        93.00963   84.27981  9.39%   901  110s
   524   241   87.92940    7  871   93.00963   84.58489  9.06%   861  116s
   596   258     cutoff   11        93.00963   84.85854  8.76%   826  121s
   656   272   89.29969    8 1040   93.00963   84.90537  8.71%   815  126s
   686   282   92.56059   14  937   93.00963   84.90537  8.71%   811  150s
H  688   232                      91.0097041   84.90537  6.71%   810  150s
   698   236     cutoff   17        91.00970   84.92462  6.69%   812  155s
H  712   236                      91.0096929   84.92462  6.69%   806  155s
   754   248   86.43035    7  869   91.00969   84.93340  6.68%   806  162s
   781   248   89.49467    9 1173   91.00969   85.17626  6.41%   799  165s
   847   274   87.93092   11  958   91.00969   85.24404  6.34%   804  172s
H  878   274                      91.0096855   85.24404  6.34%   795  172s
   885   286     cutoff   15        91.00969   85.45226  6.11%   802  176s
   926   290   90.42360   14  553   91.00969   85.52184  6.03%   805  181s
[...]
```

A lot of things are happening here! You can see the progress of the branch-and-bound tree (first 2 columns), when a heuristic has been found (the little `H` on the left), information about the best solution and best bound (columns 6-8), and much more!

So what can you do with this information? Up until now, you needed to "read" the log files, i.e., open them up and figure out what is going on by going through it line by line. *But this is about to change!*

## What is "gurobi-logtools"

In short, `gurobi-logtools` is an [open-source Python package](https://github.com/Gurobi/gurobi-logtools) written by our Gurobi Experts team that reads the log file(s) for you and makes them available as a pandas dataframe or an Excel spreadsheet. When using Python, there are also a few nifty routines that allow you to create nice plots from your log files in a few lines of code. 

All you need to do is run `pip install gurobi-logtools` and you're good to go!

# Single log file analysis

Let's start with the simplest case, reading and analysing a single log file:

In [1]:
import gurobi_logtools as glt

summary = glt.parse("data/1202-TimeLimit300-k16x240-0.log").summary()

And we're done! Now all the information in the log file is converted into a standard dataframe. Let's have a look!

In [2]:
summary

,ReadingTime,ModelName,Rows,Columns,Nonzeros,PhysicalCores,LogicalProcessors,Threads,NumConstrs,NumVars,...,LogNumber,TimeLimit (Parameter),SoftMemLimit (Parameter),LogFile (Parameter),ModelFilePath,Seed,Version,ModelFile,Model,Log
0,0.0,k16x240,256,480,960,8,16,16,256,480,...,1,300.0,125.0,1202-TimeLimit300-k16x240-0.log,/miplib2010/k16x240.mps.bz2,0,12.0.2,k16x240,k16x240,1202-TimeLimit300


Since there is just a single line in this dataframe it may be easier to view this line as a pandas Series:

In [3]:
summary.iloc[0]

ReadingTime                  0.0
ModelName                k16x240
Rows                         256
Columns                      480
Nonzeros                     960
                     ...        
Seed                           0
Version                   12.0.2
ModelFile                k16x240
Model                    k16x240
Log            1202-TimeLimit300
Name: 0, Length: 65, dtype: object

# Multiple log file analysis

While it's nice to parse a single log file into a dataframe, `gurobi-logtools` is most powerful when reading multiple log files. Then, each log file becomes a separate row in the dataframe.  Not that you can also use [regular expressions](https://en.wikipedia.org/wiki/Regular_expression) to match several files at once based on common patterns.  The argument to the `parse` function can also be a list of strings - both complete filepaths or regular expressions:

In [4]:
results = glt.parse([
    "data/1003*.log",
    "data/1202*.log",
])

summary = results.summary()

Now we have read 360 different log files and parsed their contents into a pandas DataFrame:

In [5]:
summary

,ReadingTime,ModelName,Rows,Columns,Nonzeros,PhysicalCores,LogicalProcessors,Threads,NumConstrs,NumVars,...,SoftMemLimit (Parameter),LogFile (Parameter),Presolve (Parameter),MIPFocus (Parameter),ModelFilePath,Seed,Version,ModelFile,Model,Log
0,0.0,k16x240,256,480,960,8,16,16,256,480,...,125.0,1003-MIPFocus0-Presolve1-TimeLimit300-k16x240-...,1,0,/miplib2010/k16x240.mps.bz2,0,10.0.3,k16x240,k16x240,1003-MIPFocus0-Presolve1-TimeLimit300
1,0.0,k16x240,256,480,960,8,16,16,256,480,...,125.0,1003-MIPFocus0-Presolve1-TimeLimit300-k16x240-...,1,0,/miplib2010/k16x240.mps.bz2,1,10.0.3,k16x240,k16x240,1003-MIPFocus0-Presolve1-TimeLimit300
2,0.0,k16x240,256,480,960,8,16,16,256,480,...,125.0,1003-MIPFocus0-Presolve1-TimeLimit300-k16x240-...,1,0,/miplib2010/k16x240.mps.bz2,2,10.0.3,k16x240,k16x240,1003-MIPFocus0-Presolve1-TimeLimit300
3,0.0,k16x240,256,480,960,8,16,16,256,480,...,125.0,1003-MIPFocus0-Presolve1-TimeLimit300-k16x240-...,1,0,/miplib2010/k16x240.mps.bz2,3,10.0.3,k16x240,k16x240,1003-MIPFocus0-Presolve1-TimeLimit300
4,0.0,k16x240,256,480,960,8,16,16,256,480,...,125.0,1003-MIPFocus0-Presolve1-TimeLimit300-k16x240-...,1,0,/miplib2010/k16x240.mps.bz2,4,10.0.3,k16x240,k16x240,1003-MIPFocus0-Presolve1-TimeLimit300
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
355,0.0,k16x240,256,480,960,8,16,16,256,480,...,125.0,1202-TimeLimit500-k16x240-5.log,-1,0,/miplib2010/k16x240.mps.bz2,15,12.0.2,k16x240,k16x240,1202-TimeLimit500
356,0.0,k16x240,256,480,960,8,16,16,256,480,...,125.0,1202-TimeLimit500-k16x240-6.log,-1,0,/miplib2010/k16x240.mps.bz2,16,12.0.2,k16x240,k16x240,1202-TimeLimit500
357,0.0,k16x240,256,480,960,8,16,16,256,480,...,125.0,1202-TimeLimit500-k16x240-7.log,-1,0,/miplib2010/k16x240.mps.bz2,17,12.0.2,k16x240,k16x240,1202-TimeLimit500
358,0.0,k16x240,256,480,960,8,16,16,256,480,...,125.0,1202-TimeLimit500-k16x240-8.log,-1,0,/miplib2010/k16x240.mps.bz2,18,12.0.2,k16x240,k16x240,1202-TimeLimit500


# Visualizations

Now that we have all this data at our (pythonic) fingertips, we can start doing all sorts of analysis, much of which is best done with the help of charts.

The `glt.plot()` function combines the power of dashboards through [`ipywidgets`](https://ipywidgets.readthedocs.io/en/stable/) and plotting functions from [`plotly.express`](https://plotly.com/python/plotly-express/). To get started, you only need to run `glt.plot(summary)` to get a dashboard where you can explore your data and results!

In [ ]:
glt.plot(summary)

![default plot](assets/plot_default.png)

All available columns in the DataFrame can be selected for plotting.  Additionally, we add a columns called **Parameters** which provides an aesthetic label for parameter sets.

All interactive parameters can also be set upfront to get the desired figure right away:

In [ ]:
glt.plot(
    summary,
    x="Runtime",
    y="Parameters",
    type="box", 
    color="Version", 
    sort_field="Runtime",
    sort_axis="y",
    sort_metric="mean",
    log_x=True, 
    title="Gurobi Performance",
    show_legend=True,
    height=500,
)

![plot with arguments](assets/plot_with_arguments.png)

# One more thing... progress reports (also known as "timelines")!

So far, we have demonstrated exploring the "final" results of the log file, such as Runtime and MIP gap.  But `gurobi-logtools` actually can parse entire log files and report on progress of

- the [No Relaxation (NoRel) heuristic](https://support.gurobi.com/hc/en-us/articles/30992739009553-How-do-I-use-the-No-Relaxation-NoRel-heuristic)
- any pre-tree solutions found, i.e. those found by heuristics prior to the branch and bound tree (but not including those found by NoRel)
- the root linear relaxation
- the node log, i.e. the section of log related to the branch and bound tree

Each of the four items listed above corresponds to a (possibly empty) pandas DataFrame, that can be retreived via the `ParseResult.progress` function.

For example:

In [8]:
results.progress("nodelog")

,CurrentNode,RemainingNodes,Obj,Depth,IntInf,Incumbent,BestBd,Gap,ItPerNode,Time,NewSolution,Pruned,LogFilePath,LogNumber,ModelFilePath,Seed,Version,ModelFile,Model,Log
0,0,0.0,2769.83800,0.0,14.0,23615.0,2769.83800,0.8830,NaN,0.0,NaN,NaN,data/1003-MIPFocus0-Presolve1-TimeLimit300-k16...,1,/miplib2010/k16x240.mps.bz2,0,10.0.3,k16x240,k16x240,1003-MIPFocus0-Presolve1-TimeLimit300
1,0,0.0,NaN,NaN,NaN,11359.0,2769.83800,0.7560,NaN,0.0,H,NaN,data/1003-MIPFocus0-Presolve1-TimeLimit300-k16...,1,/miplib2010/k16x240.mps.bz2,0,10.0.3,k16x240,k16x240,1003-MIPFocus0-Presolve1-TimeLimit300
2,0,0.0,NaN,NaN,NaN,11195.0,2769.83800,0.7530,NaN,0.0,H,NaN,data/1003-MIPFocus0-Presolve1-TimeLimit300-k16...,1,/miplib2010/k16x240.mps.bz2,0,10.0.3,k16x240,k16x240,1003-MIPFocus0-Presolve1-TimeLimit300
3,0,0.0,NaN,NaN,NaN,11074.0,8084.69631,0.2700,NaN,0.0,H,NaN,data/1003-MIPFocus0-Presolve1-TimeLimit300-k16...,1,/miplib2010/k16x240.mps.bz2,0,10.0.3,k16x240,k16x240,1003-MIPFocus0-Presolve1-TimeLimit300
4,0,0.0,8084.69631,0.0,26.0,11074.0,8084.69631,0.2700,NaN,0.0,NaN,NaN,data/1003-MIPFocus0-Presolve1-TimeLimit300-k16...,1,/miplib2010/k16x240.mps.bz2,0,10.0.3,k16x240,k16x240,1003-MIPFocus0-Presolve1-TimeLimit300
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12489,42,64.0,NaN,NaN,NaN,10678.0,9171.69176,0.1410,29.3,0.0,H,NaN,data/1202-TimeLimit500-k16x240-9.log,1,/miplib2010/k16x240.mps.bz2,19,12.0.2,k16x240,k16x240,1202-TimeLimit500
12490,1985,1332.0,NaN,NaN,NaN,10674.0,9442.57417,0.1150,17.4,0.0,H,NaN,data/1202-TimeLimit500-k16x240-9.log,1,/miplib2010/k16x240.mps.bz2,19,12.0.2,k16x240,k16x240,1202-TimeLimit500
12491,40113,13059.0,10429.54950,41.0,10.0,10674.0,10107.98350,0.0530,27.6,5.0,NaN,NaN,data/1202-TimeLimit500-k16x240-9.log,1,/miplib2010/k16x240.mps.bz2,19,12.0.2,k16x240,k16x240,1202-TimeLimit500
12492,79292,14089.0,10399.90100,40.0,19.0,10674.0,10304.20260,0.0346,29.3,10.0,NaN,NaN,data/1202-TimeLimit500-k16x240-9.log,1,/miplib2010/k16x240.mps.bz2,19,12.0.2,k16x240,k16x240,1202-TimeLimit500


Since NoRel was not used in the optimizations recorded by the logs in our dataset the result for NoRel will be an empty DataFrame:

In [9]:
results.progress("norel")

,LogFilePath,LogNumber,ModelFilePath,Seed,Version,ModelFile,Model,Log


Each optimization corresponding to the log files in the dataset has a single solution found during the presolve routine.  Consequently the DataFrame for the "Pre-tree solutions" will have 180 lines - one for each log file.  If multiple solutions were found prior to the branch and bound tree then we would have even more rows.

In [10]:
results.progress("pretreesols")

,Incumbent,LogFilePath,LogNumber,ModelFilePath,Seed,Version,ModelFile,Model,Log
0,23615.0,data/1003-MIPFocus0-Presolve1-TimeLimit300-k16...,1,/miplib2010/k16x240.mps.bz2,0,10.0.3,k16x240,k16x240,1003-MIPFocus0-Presolve1-TimeLimit300
1,18783.0,data/1003-MIPFocus0-Presolve1-TimeLimit300-k16...,1,/miplib2010/k16x240.mps.bz2,1,10.0.3,k16x240,k16x240,1003-MIPFocus0-Presolve1-TimeLimit300
2,20929.0,data/1003-MIPFocus0-Presolve1-TimeLimit300-k16...,1,/miplib2010/k16x240.mps.bz2,2,10.0.3,k16x240,k16x240,1003-MIPFocus0-Presolve1-TimeLimit300
3,20220.0,data/1003-MIPFocus0-Presolve1-TimeLimit300-k16...,1,/miplib2010/k16x240.mps.bz2,3,10.0.3,k16x240,k16x240,1003-MIPFocus0-Presolve1-TimeLimit300
4,20422.0,data/1003-MIPFocus0-Presolve1-TimeLimit300-k16...,1,/miplib2010/k16x240.mps.bz2,4,10.0.3,k16x240,k16x240,1003-MIPFocus0-Presolve1-TimeLimit300
...,...,...,...,...,...,...,...,...,...
355,16199.0,data/1202-TimeLimit500-k16x240-5.log,1,/miplib2010/k16x240.mps.bz2,15,12.0.2,k16x240,k16x240,1202-TimeLimit500
356,20319.0,data/1202-TimeLimit500-k16x240-6.log,1,/miplib2010/k16x240.mps.bz2,16,12.0.2,k16x240,k16x240,1202-TimeLimit500
357,21882.0,data/1202-TimeLimit500-k16x240-7.log,1,/miplib2010/k16x240.mps.bz2,17,12.0.2,k16x240,k16x240,1202-TimeLimit500
358,19474.0,data/1202-TimeLimit500-k16x240-8.log,1,/miplib2010/k16x240.mps.bz2,18,12.0.2,k16x240,k16x240,1202-TimeLimit500


The log files in our dataset also do not contain and logging related to the root LP - the root LP solves so quickly (0 seconds!) that lines related to the progress of the LP solve are logged.

```Plain
Presolve time: 0.00s
Presolved: 256 rows, 480 columns, 960 nonzeros
Variable types: 240 continuous, 240 integer (240 binary)

Root relaxation: objective 2.769838e+03, 51 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 2769.83800    0   14 23615.0000 2769.83800  88.3%     -    0s
```

This results in an empty DataFrame:

In [11]:
results.progress("rootlp")

,LogFilePath,LogNumber,ModelFilePath,Seed,Version,ModelFile,Model,Log


This additional information allows for visualizations of the **solving progress**, for example the primal/dual bound improvement:

In [ ]:
nodelog = results.progress("nodelog")
glt.plot(
    nodelog[nodelog["Log"] == "1202-MIPFocus1-Presolve2-TimeLimit300"],
    title="1202-MIPFocus1-Presolve2-TimeLimit300",
    type="line",
    x="Time",
    y="Gap",
    color="Seed",
    log_y=True,
)

![nodelog plot](assets/plot_nodelog.png)

# The sky is the limit - customizing your plots

Since `gurobi-logtools` simply returns pandas dataframes, you can use all the standard Python machinery to draw more customizable plots. We like `plotly`, so here are a couple of examples of using the fully featured `plotly` library. First, let's have a look at an example of plotting the primal and dual bound progress using filled charts:

In [ ]:
import plotly.graph_objects as go

default_run = nodelog[(nodelog["Log"] == "1202-MIPFocus1-Presolve2-TimeLimit300") & (nodelog["Seed"] == 0)]

fig = go.Figure()
fig.add_trace(go.Scatter(x=default_run["Time"], y=default_run["Incumbent"], name="Primal Bound"))
fig.add_trace(go.Scatter(x=default_run["Time"], y=default_run["BestBd"], fill="tonexty", name="Dual Bound"))
fig.update_xaxes(title_text="Runtime")
fig.update_yaxes(title_text="Objective function value")
fig.show()

![plot bounds single](assets/plot_bounds_single.png)

Remember that run with the very different run times using `Cuts=0` and `Heuristics=0.1`? Let's have a closer look at that:

In [ ]:
from plotly.subplots import make_subplots

selected_run = nodelog[(nodelog["Log"] == "1202-MIPFocus1-Presolve2-TimeLimit300")]
fastest_run = selected_run[selected_run["Seed"] == 0]
slowest_run = selected_run[selected_run["Seed"] == 1]

fig = make_subplots(rows=2, cols=1, shared_xaxes=True)
fig.append_trace(
    go.Scatter(
        x=slowest_run["Time"], y=slowest_run["Incumbent"], name="Primal Bound - Seed 1"
    ),
    row=1,
    col=1,
)
fig.append_trace(
    go.Scatter(
        x=slowest_run["Time"],
        y=slowest_run["BestBd"],
        fill="tonexty",
        name="Dual Bound - Seed 1",
    ),
    row=1,
    col=1,
)

fig.append_trace(
    go.Scatter(
        x=fastest_run["Time"], y=fastest_run["Incumbent"], name="Primal Bound - Seed 0"
    ),
    row=2,
    col=1,
)
fig.append_trace(
    go.Scatter(
        x=fastest_run["Time"],
        y=fastest_run["BestBd"],
        fill="tonexty",
        name="Dual Bound - Seed 0",
    ),
    row=2,
    col=1,
)

fig.update_layout(title_text="Comparing bound progression between 2 random seeds", height=500, width=1000)
fig.update_yaxes(range=[8000, 12000], row=1, col=1)
fig.update_yaxes(range=[8000, 12000], row=2, col=1)

fig.update_xaxes(title_text="Runtime")
fig.update_yaxes(title_text="Objective function value")

fig.show()

![plot bounds comparison](assets/plot_bounds_comparison.png)

# One more thing... Excel output!

If you prefer working in Excel rather than in Python to explore your data, you can also directly create Excel worksheets from your log files using `gurobi_logtools` as a command-line tool:

In [15]:
! python -m gurobi_logtools analysis.xlsx data/*.log

extracted 360 log(s) to analysis.xlsx


Create additional progress tables in separate worksheets using option `--progress`.

Run `python -m gurobi_logtools --help` for a list of all available options